In [1]:
import time, os, json, PyPDF2, re
import pandas as pd

# PDF -> Text

In [2]:
def read_pdf2017(pdf_path, r, name):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        texts = [i.extract_text().replace('\t', ' ') for i in reader.pages]
    clean = texts[0].split('Location Information')[0]
    #print(clean)
    txt_temp = ''
    code = ''
    lastcode = ''
    for i in texts:
        sent = i.replace(clean, '')
        for k in sent.split('\n'):
            if 'ITEM' in k: 
                code = k.split(':')[0]
                code = ' '.join(code.split())
                if '.' in code: code = code.split('.')[-1]
                #print('----', k)
                #if lastcode == 'ITEM 3': print('-----', txt_temp)
                if lastcode not in r: r[lastcode] = []
                if len(txt_temp)!=0:
                    r[lastcode].append(name + '\n\n\n' + txt_temp)
                    txt_temp = ''
                    lastcode = code
            else:
                txt_temp = txt_temp+'\n'+k

In [3]:
pdf_directory = '/Users/bjcliang/Desktop/Nate/Census/Surveys2017'
record = {'':[]}

for pdf_file in os.listdir(pdf_directory):
    if pdf_file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_directory, pdf_file)
        print(pdf_path, end = '\r')
        read_pdf2017(pdf_path, record, pdf_file)
with open('/Users/bjcliang/Desktop/Nate/Census/Surveys2017/rec.json', 'w') as json_file:
    json.dump(record, json_file)

# Clean Data

In [1]:
import time, os, json, PyPDF2, re, docx
import pandas as pd

In [25]:
def general_process(text):
    def replace_repeated_num(text):
        pattern = r'(\b\d\.\s)+'
        def replacement(match):
            number = match.group(1).strip()
            return f'\n{number}'
        return re.sub(pattern, replacement, text)
    
    def replace_repeated_string(text):
        pattern = r'(\b[A-Za-z\s]+[A-Za-z]+\.\s)\1+'

        def replacement(match):
            # Extract the first occurrence
            first_occurrence = match.group(1)
            return f'{first_occurrence.strip()}\n'
        return re.sub(pattern, replacement, text)
    
    
    text = re.sub(r'\s+', ' ', text)
    text = replace_repeated_num(text)
    text = replace_repeated_string(text)
    text = re.sub('\n', ' ', text)
    pattern = r'\n.*\.pdf Generated at \d{4}-\d{2}-\d{2} \d{2}:\d{2} [APM]{2}.*\n'
    text = re.sub(pattern, '', text)
    text = text.split('Do Not Submit')[0]
    #pattern = r'\b(?!ZIP|EIN|CFN|IRS|LONG|DISTANCE|LOCAL|TOTAL)([A-Z]{2,}\b(?:\s[A-Z]{2,}\b)*)'
    #text = re.sub(pattern, r'\n\1', text)
    text = text.split('Select ALL that apply.')[0]
    text = text.split('Description Description Value Value Product Code Product Code')[0]
    text = text.split('Select only ONE.')[0]
    
    text = re.sub('INSTRUCTIONS FOR ADMINISTRATIVE EXPENSES', '\nINSTRUCTIONS FOR ADMINISTRATIVE EXPENSES ', text)
    pattern = r'(\b\w+\. )(\1)+'
    text = re.sub(pattern, r'\1', text)
    pattern = r'(\d+\.\s*[A-Z])'
    text = re.sub(pattern, r'\n\1', text)
    
    sp = 'HOW TO REPORT DOLLAR FIGURES'
    text.replace(find_variations_with_spaces(sp, text)[0], sp)
    
    return text.strip()

In [3]:
record = json.load(open('/Users/bjcliang/Desktop/Nate/Census/Surveys2017/rec.json'))
record.keys()

dict_keys(['', 'ITEM 1', 'ITEM 3', 'ITEM 4', 'ITEM 5', 'ITEM 7', 'ITEM 8', 'ITEM 16', 'ITEM 17', 'ITEM 20', 'ITEM 22', 'ITEM 31', 'ITEM 2', 'ITEM 25', 'ITEM 26', 'ITEM 28', 'ITEM 32', 'ITEM 33', 'ITEM 33A', 'ITEM 33B', 'ITEM 9', 'ITEM 10', 'ITEM 17A', 'ITEM 17B', 'ITEM 27', 'ITEM 5A', 'ITEM 5B'])

In [4]:
rec_new = dict()
for item in record:
    if item == '': pass
        #rec_new['Introduction'] = dict()
        #for txt in record[item]:
            #name, t = txt.split('.pdf\n\n\n')
            #t = general_process(t)
            #print(t)
            #if t not in rec_new['Introduction']: rec_new['Introduction'][t] = []
            #rec_new['Introduction'][t].append(name)
    else:
        pattern = r'(\d+)[A-Za-z]+'
        superitem = re.sub(pattern, r'\1', item)
        if superitem not in rec_new: rec_new[superitem] = dict()
            
        
        #if item not in rec_new[superitem]: rec_new[superitem][item] = dict()
        for txt in record[item]:
            #print(txt)
            name, t = txt.split('.pdf\n\n\n')
            t = item + ': ' + general_process(t)
            if t not in rec_new[superitem]: rec_new[superitem][t] = []
            rec_new[superitem][t].append(name)

In [11]:
from docx.oxml.ns import qn
from docx.shared import Pt

def create_word_document_from_dict(data, path):
    doc = docx.Document()
    doc.add_heading('Surveys 2017', level=1)
    
    for superitem_index, (superitem, items) in enumerate(data.items(), start=1):
        doc.add_heading(f'{superitem_index}. {superitem}', level=2)
        
        for item_index, (item, details) in enumerate(items.items(), start=1):
            
            #p = doc.add_paragraph(style='ListBullet')
            #sentences = item.split('. ')
            #for sentence in sentences:
            #    if sentence.endswith('?'):
            #        run = p.add_run(sentence + '. ')
            #        run.font.highlight_color = docx.enum.text.WD_COLOR_INDEX.YELLOW
            #    else:
            #        p.add_run(sentence + '. ')
            
            doc.add_paragraph(item).style='ListBullet'
            
            surveys = ', '.join(details)
            p = doc.add_paragraph()
            p = p.add_run(f'Surveys included: {surveys}')
            p.font.size = Pt(9)
            #p.italic = True   

    # Save the document
    doc.save(path)



In [12]:
create_word_document_from_dict(rec_new, 'Economic Census Surveys 2017.docx')